In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Sample data for word prediction
word_data = "Hello world. I am learning deep learning. I love neural networks."

# Word prediction
tokenizer = Tokenizer()
tokenizer.fit_on_texts([word_data])
word_total_words = len(tokenizer.word_index) + 1

# Convert text to sequence of tokens
input_sequences = []
for line in word_data.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len_word = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len_word, padding='pre')

X_word, y_word = input_sequences[:,:-1], input_sequences[:,-1]
y_word = to_categorical(y_word, num_classes=word_total_words)

# Define the RNN model for word prediction
model_word = Sequential()
model_word.add(Embedding(word_total_words, 100, input_length=max_sequence_len_word-1))
model_word.add(SimpleRNN(150, return_sequences=True))
model_word.add(SimpleRNN(100))
model_word.add(Dense(word_total_words, activation='softmax'))
model_word.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_word.summary()

# Train the word prediction model
model_word.fit(X_word, y_word, epochs=100, verbose=2)

# Word prediction function
def predict_next_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=max_sequence_len_word-1, padding='pre')
    prediction = model.predict(sequence, verbose=0)
    predicted_index = np.argmax(prediction, axis=-1)
    predicted_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            predicted_word = word
            break
    return predicted_word

# Test word prediction
print("Word prediction:")
print(predict_next_word(model_word, tokenizer, "I am learning"))





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            1000      
                                                                 
 simple_rnn (SimpleRNN)      (None, 4, 150)            37650     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               25100     
                                                                 
 dense (Dense)               (None, 10)                1010      
                                                                 
Total params: 64760 (252.97 KB)
Trainable params: 64760 (252.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100


1/1 - 2s - loss: 2.3257 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/100
1/1 - 0s - loss: 2.1560 - accuracy: 0.2500 - 5ms/epoc